# Agentic RAG

- Step 1: Use a base LLM to generate content in the style of an author
- Step 2: Do this using RAG
- Step 3: Do this using agentic RAG

## Basic Retrieval with DuckDuckGo

Let's build a simple agent that can search the web using DuckDuckGo. This agent will retrieve information and synthesize responses to answer queries. With Agentic RAG, our agent can:

- Search for articles from an author
- Refine results to include relevant pieces
- Synthesize information into a take

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool
from smolagents.models import LiteLLMModel

# Initialize the search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the model
local_model = LiteLLMModel(model_id="ollama_chat/qwen3:8b")

agent = CodeAgent(
    model = local_model,
    tools=[search_tool]
)

In [ ]:
AUTHOR = "Robert Wright"
HEADLINE = "US Strikes Iran’s Nuclear Sites, Risking Wider War in Mideast"
STORY = """
The US carried out airstrikes on three nuclear sites in Iran overnight, directly entering Israel’s war with Tehran despite President Donald Trump’s longtime promises to avoid new foreign conflicts.
Trump said Iran’s key nuclear enrichment facilities had been “totally obliterated” and warned of “far greater” attacks unless the Islamic Republic agreed to make peace, raising the prospect of deeper US involvement in a Middle East war sparked by Israeli strikes nine days ago.
Iranian officials said the ongoing attacks by Israel — now joined by the US — had left little room for diplomacy, arguing that negotiations are impossible while the country is under assault. Tehran fired missiles at Israel in response but has so far stopped short of targeting American forces or assets in the region.
American B-2 bombers dropped a dozen of the 30,000-pound (13,600-kilogram) bunker-buster bombs on Fordow, a uranium-enrichment site buried deep under a mountain, the New York Times reported. Natanz and Isfahan, two other nuclear facilities, were also struck using similar weapons and cruise missiles.
“Our objective was the destruction of Iran’s nuclear enrichment capacity and a stop to the nuclear threat posed by the world’s No. 1 state sponsor of terror,” Trump said. “Iran, the bully of the Middle East, must now make peace. If they do not, future attacks will be far greater — and a lot easier.”
Iran’s foreign minister, Abbas Araghchi, said the American strikes are “outrageous and will have everlasting consequences.” 
"""

In [ ]:
prompt = f"""
Use the following news headline and story to generate an output or take that you think would most likely come from the author {AUTHOR}.
Headline: {HEADLINE}
Story: {STORY}
"""

In [ ]:
response = agent.run(prompt)
print(response)

In [ ]:
import os
import re
from typing import List, Dict, Any
from pathlib import Path

from smolagents import CodeAgent, DuckDuckGoSearchTool, Tool
from smolagents.models import LiteLLMModel
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# RTF file reader utility
def read_rtf_file(file_path: str) -> str:
    """
    Simple RTF reader that extracts plain text from RTF files.
    For production use, consider using python-rtf or striprtf libraries.
    """
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()
        
        # Basic RTF cleaning - remove RTF control codes
        # This is a simplified approach; for complex RTF files, use dedicated libraries
        content = re.sub(r'\\[a-z]+\d*', '', content)  # Remove RTF commands
        content = re.sub(r'[{}]', '', content)  # Remove braces
        content = re.sub(r'\s+', ' ', content)  # Normalize whitespace
        content = content.strip()
        
        return content
    except Exception as e:
        print(f"Error reading RTF file {file_path}: {e}")
        return ""

# Step 1: Base LLM Content Generation
class BaseAuthorGenerator:
    """Step 1: Basic author style generation without RAG"""
    
    def __init__(self, author_name: str, model_id: str = "ollama_chat/qwen3:8b"):
        self.author_name = author_name
        self.model = LiteLLMModel(model_id=model_id)
        self.agent = CodeAgent(model=self.model, tools=[])
    
    def generate_response(self, headline: str, story: str) -> str:
        prompt = f"""
        You are tasked with generating content in the style of {self.author_name}.
        
        Based on the following news headline and story, create a response or commentary 
        that captures {self.author_name}'s distinctive voice, perspective, and style.
        
        Headline: {headline}
        Story: {story}
        
        Generate a response that {self.author_name} would likely write:
        """
        
        return self.agent.run(prompt)

# Step 2: RAG-Enhanced Author Generator
class AuthorDocumentRetrieverTool(Tool):
    """Custom retriever tool for author's documents"""
    
    name = "author_document_retriever"
    description = "Retrieves relevant passages from the author's existing works to inform style and content generation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to search for relevant content in the author's documents.",
        }
    }
    output_type = "string"

    def __init__(self, docs: List[Document], **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=3  # Retrieve top 3 most relevant passages
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        
        docs = self.retriever.invoke(query)
        
        if not docs:
            return "No relevant passages found in the author's documents."
        
        result = "\n=== RELEVANT AUTHOR PASSAGES ===\n"
        for i, doc in enumerate(docs, 1):
            result += f"\n--- Passage {i} ---\n{doc.page_content}\n"
        
        return result

class RAGAuthorGenerator:
    """Step 2: RAG-enhanced author style generation"""
    
    def __init__(self, author_name: str, rtf_documents_path: str, model_id: str = "ollama_chat/qwen3:8b"):
        self.author_name = author_name
        self.model = LiteLLMModel(model_id=model_id)
        
        # Load and process author's documents
        self.documents = self._load_author_documents(rtf_documents_path)
        
        # Create retriever tool
        self.retriever_tool = AuthorDocumentRetrieverTool(self.documents)
        
        # Initialize agent with retriever
        self.agent = CodeAgent(
            model=self.model,
            tools=[self.retriever_tool]
        )
    
    def _load_author_documents(self, documents_path: str) -> List[Document]:
        """Load and process RTF documents from the specified path"""
        documents = []
        path = Path(documents_path)
        
        if not path.exists():
            print(f"Warning: Path {documents_path} does not exist. Creating empty document set.")
            return documents
        
        # Process RTF files
        for rtf_file in path.glob("*.rtf"):
            content = read_rtf_file(str(rtf_file))
            if content:
                documents.append(Document(
                    page_content=content,
                    metadata={"source": rtf_file.name, "file_path": str(rtf_file)}
                ))
        
        # Split documents into chunks for better retrieval
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            add_start_index=True,
            strip_whitespace=True,
            separators=["\n\n", "\n", ".", "!", "?", " ", ""],
        )
        
        processed_docs = text_splitter.split_documents(documents)
        print(f"Loaded {len(documents)} documents, split into {len(processed_docs)} chunks")
        
        return processed_docs
    
    def generate_response(self, headline: str, story: str) -> str:
        prompt = f"""
        You are generating content in the style of {self.author_name}.
        
        First, use the author_document_retriever tool to find relevant passages from {self.author_name}'s 
        existing works that relate to the topic, themes, or style needed for this response.
        
        Search for passages related to: "{headline}"
        
        Then, based on the retrieved passages and the following news content, generate a response 
        that authentically captures {self.author_name}'s voice, style, and perspective:
        
        Headline: {headline}
        Story: {story}
        
        Make sure to incorporate the stylistic elements and thematic approaches found in the retrieved passages.
        """
        
        return self.agent.run(prompt)

# Step 3: Advanced Agentic RAG with Multiple Tools
class AdvancedAgenticRAG:
    """Step 3: Full agentic RAG with multiple tools and sophisticated reasoning"""
    
    def __init__(self, author_name: str, rtf_documents_path: str, model_id: str = "ollama_chat/qwen3:8b"):
        self.author_name = author_name
        self.model = LiteLLMModel(model_id=model_id)
        
        # Load author documents
        self.documents = self._load_author_documents(rtf_documents_path)
        
        # Initialize tools
        self.retriever_tool = AuthorDocumentRetrieverTool(self.documents)
        self.search_tool = DuckDuckGoSearchTool()
        
        # Initialize agent with multiple tools
        self.agent = CodeAgent(
            model=self.model,
            tools=[self.retriever_tool, self.search_tool]
        )
    
    def _load_author_documents(self, documents_path: str) -> List[Document]:
        """Load and process RTF documents"""
        documents = []
        path = Path(documents_path)
        
        if not path.exists():
            print(f"Warning: Path {documents_path} does not exist. Creating empty document set.")
            return documents
        
        for rtf_file in path.glob("*.rtf"):
            content = read_rtf_file(str(rtf_file))
            if content:
                documents.append(Document(
                    page_content=content,
                    metadata={"source": rtf_file.name, "file_path": str(rtf_file)}
                ))
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            add_start_index=True,
            strip_whitespace=True,
            separators=["\n\n", "\n", ".", "!", "?", " ", ""],
        )
        
        processed_docs = text_splitter.split_documents(documents)
        print(f"Loaded {len(documents)} documents, split into {len(processed_docs)} chunks")
        
        return processed_docs
    
    def generate_response(self, headline: str, story: str) -> str:
        prompt = f"""
        You are an expert at generating content in the authentic style of {self.author_name}.
        
        Your task is to create a thoughtful response to the following news story that captures 
        {self.author_name}'s distinctive voice, analytical approach, and perspective.
        
        To do this effectively, you should:
        
        1. First, use the author_document_retriever tool to find relevant passages from {self.author_name}'s 
           existing works that relate to the themes, topics, or issues in this news story.
        
        2. If you need additional context or recent information about the topic, use the duckduckgo_search 
           tool to gather more current information.
        
        3. Analyze the retrieved content to identify key stylistic elements, argumentative patterns, 
           and thematic approaches that characterize {self.author_name}'s work.
        
        4. Generate a response that authentically reflects {self.author_name}'s voice while addressing 
           the current news story.
        
        News to respond to:
        Headline: {headline}
        Story: {story}
        
        Create a response that {self.author_name} would likely write, incorporating their distinctive 
        style, perspective, and approach to similar topics based on their existing work.
        """
        
        return self.agent.run(prompt)

# Example usage and testing
def main():
    # Configuration
    AUTHOR = "Your Author Name"  # Replace with actual author name
    HEADLINE = "Tech Giants Announce New AI Regulations"  # Replace with actual headline
    STORY = "Major technology companies have agreed to new self-imposed regulations..."  # Replace with actual story
    RTF_DOCUMENTS_PATH = "./author_documents"  # Path to your RTF files
    
    print("=== AGENTIC RAG PROGRESSION ===\n")
    
    # Step 1: Base LLM Generation
    print("STEP 1: Base LLM Content Generation")
    print("-" * 40)
    base_generator = BaseAuthorGenerator(AUTHOR)
    step1_response = base_generator.generate_response(HEADLINE, STORY)
    print(f"Response: {step1_response}\n")
    
    # Step 2: RAG-Enhanced Generation
    print("STEP 2: RAG-Enhanced Generation")
    print("-" * 40)
    rag_generator = RAGAuthorGenerator(AUTHOR, RTF_DOCUMENTS_PATH)
    step2_response = rag_generator.generate_response(HEADLINE, STORY)
    print(f"Response: {step2_response}\n")
    
    # Step 3: Advanced Agentic RAG
    print("STEP 3: Advanced Agentic RAG")
    print("-" * 40)
    advanced_rag = AdvancedAgenticRAG(AUTHOR, RTF_DOCUMENTS_PATH)
    step3_response = advanced_rag.generate_response(HEADLINE, STORY)
    print(f"Response: {step3_response}\n")

if __name__ == "__main__":
    main()